# Legendre transformation and Gibbs energy

In order to enable fatigue load scenarios, this notebook shows how to transform a model derived based on the Helmholtz free energy to the Gibbs free energy 

Given the Helmholtz free energy controlled by strain/displacement $u$ and temperature $\vartheta$
$$
F = \hat{F}(u, \vartheta; \mathcal{E})
$$
we seek the complementary form of the state representaiton that can is expressed using the stress level $\sigma$ instead of the strain $u$   
$$
G = \hat{G}(\sigma, \vartheta; \mathcal{E})
$$
The Legendre transform exploits the fact that 
$$
\hat{F}(u, \vartheta; \mathcal{E})
 + \hat{G}(\sigma, \vartheta; \mathcal{E}) = \sigma u. 
$$

The generic derivation below exploits the fact that stress level definition as the rate of change of $F$ due to change of strain is obtained as the gradient
$$
\sigma = \partial_u F = \Sigma(u) 
$$
which can be symbolically inverted to obtain
$$
u = \Sigma^{-1}(\sigma).
$$

Thus the substitution into the Legendre transform renders
$$
\hat{G}(\sigma, \vartheta; \mathcal{E}) = \sigma \Sigma^{-1}(\sigma) - \hat{F}(\Sigma^{-1}(\sigma), \vartheta; \mathcal{E})
$$

In the framework below, two instances of the `GSM` class are constructed. The first is based on Helmholtz energy and the second on the Gibbs energy. The dissipative potential and internal variables are the same. The only difference is the swapped meaning of the variables `u` and `sig`. In case of Gibbs based GSM, input variable `u` means stress and output variable `sig` means strain. This shall be better formalized in future implementation of the API.

In [ ]:
%matplotlib widget
from bmcs_matmod import GSM
import matplotlib.pylab as plt 
import sympy as sp
import numpy as np
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_d_nikh import Potential1D_T_E_VP_D_NIKH_SymbExpr
sp.init_printing()

## Gibbs free energy

In [ ]:
p1d = Potential1D_T_E_VP_D_NIKH_SymbExpr()
eps_a = p1d.eps_a
sig_a = p1d.sig_a
dF_du = p1d.F_.diff(eps_a)
# dF_du = dF_du.xreplace({h: 0 for h in dF_du.atoms(sp.DiracDelta)})
# dF_du = dF_du.xreplace({h: 1 for h in dF_du.atoms(sp.Heaviside)})
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, eps_a, dF_du)])
subs_u_sig_ = dict(zip(eps_a, u_sig_))
subs_u_sig_

In [ ]:
sig_x_u_ = (sig_a.T * eps_a)[0]
G_expr = sig_x_u_ - p1d.F_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

In [ ]:
gsm_F = GSM(
    name = 'gsm_F_1d_t_e_vp_d_nikh',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = (-1, 1, 1, -1),
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

gsm_G = GSM(
    name = 'gsm_G_1d_t_e_vp_d_nikh',
    u_vars = p1d.sig_a,
    sig_vars = p1d.eps_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = (1, -1, -1, 1),
    F_expr = G_,
    dF_sign = -1,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

In [ ]:
gsm_F.save_to_disk()
gsm_G.save_to_disk()

## Review input and output expressions

In [ ]:
gsm_G.Eps_vars, gsm_F.Eps_vars

In [ ]:
gsm_F.F_expr

In [ ]:
gsm_F.phi_

In [ ]:
gsm_G.F_expr

In [ ]:
sp.simplify(gsm_F.dF_dEps_), sp.simplify(gsm_G.dF_dEps_)

In [ ]:
gsm_F.Sig_vars, gsm_F.Sig_, sp.simplify(gsm_G.Sig_)

In [ ]:
sp.simplify(gsm_F.df_dSig_), sp.simplify(gsm_G.df_dSig_)

In [ ]:
sp.simplify(gsm_F.Phi_), sp.simplify(gsm_G.Phi_)

In [ ]:
sp.simplify(gsm_F.df_dEps_), sp.simplify(gsm_G.df_dEps_)

In [ ]:
gsm_G.phi_, sp.simplify(gsm_G.Phi_)

In [ ]:
gsm_F.dSig_dEps_, sp.simplify(gsm_G.dSig_dEps_)

In [ ]:
sig_ = gsm_F.F_expr.diff(gsm_F.u_vars)
sig_

## Monotonic loading

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_KH_factor = 4
_KH = _E * _KH_factor
_K_ratio = 0.01 # 0.015
_K = _KH * _K_ratio
_H = _KH * (1 - _K_ratio)
material_params = dict(
    E_=_E, 
    gamma_lin_= _H, # _E * 10, 
    gamma_exp_= 0.1, # _E * 10, 
    alpha_0_= 10, # _E * 10, 
    K_lin_= _K, # _E / 5,
    k_exp_= 0.1,
    z_0_=10,
    S_=0.008,
    c_=2.5,
    r_=2.7,
    f_c_=_f_s,
    X_0_=_X_0,  
    eta_=500,
    T_0_=20,
    C_v_=0.01, # 0.0001, 
    beta_=0.0001,
    alpha_therm_=0, # 1.2e-5,
    d_N_ = 1
)

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t, lam_t = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t 

gsm_F.vp_on = True
gsm_F.update_at_k = False
gsm_G.vp_on = True
gsm_G.update_at_k = False

In [ ]:
# params
n_t = 151
n_I = 1
u_T_max = 0.005
t_t = np.linspace(0, 1, n_t)
u_ta_F = (u_T_max * t_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, lam_t = gsm_run(gsm_F, u_ta_F, T_t, t_t, **material_params)
_max_sig = np.max(_sig_atI_F)
_max_sig
_arg_t_F = _t_t_F[np.argmax(_sig_atI_F)]
_t_F_scale = _arg_t_F * _t_t_F[-1]
_t_F_scale

In [ ]:
# params
t_t = np.linspace(0, 1, n_t)
u_ta_G = (_max_sig * t_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_G, _u_atI_G, _sig_atI_G, _T_t_G, _Eps_btI_G, _Sig_btI_G, _dF_dEps_btI_G, lam_t = gsm_run(gsm_G, u_ta_G, T_t, t_t, **material_params)
_u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_G.Eps_as_blocks(_Eps_btI_F)
_, _Z_atI, _X_atI, _Y_atI = gsm_G.Eps_as_blocks(_Sig_btI_F)


In [ ]:
from scipy.integrate import cumtrapz
fig, ((ax, ax_T, ax_Diss), (ax_omega, ax_3, ax_4)) = plt.subplots(2,3, figsize=(12,6), tight_layout=True)
ax.plot(_u_atI_F[0, :, 0], _sig_atI_F[0, :, 0], label='Helmholtz');
ax.plot(_sig_atI_G[0, :, 0], _u_atI_G[0, :, 0], label='Gibbs');
# ax_T.plot(_t_t_F, _T_t_F);
# ax_T.plot(_t_t_G * _t_F_scale, _T_t_G);
ax.legend()
ax.set_title(r'stress-strain')
ax.set_ylabel(r'$\varsigma$')
ax.set_xlabel(r'$\varepsilon$')

ax_T.plot(_u_atI_F[0, :, 0], _T_t_F, label='Helmholtz');
ax_T.plot(_sig_atI_G[0, :, 0], _T_t_G, label='Gibbs');
ax_T.legend()
ax_T.set_title(r'temperature')
ax_T.set_ylabel(r'$\vartheta$')
ax_T.set_xlabel(r'$\varepsilon$')

Diss_btI_F = cumtrapz(_dF_dEps_btI_F, _Eps_btI_F, initial=0, axis=1)
ax_Diss.plot(_t_t_F, np.sum(Diss_btI_F[...,0], axis=0), alpha=1, label='F')
Diss_btI_G = cumtrapz(_dF_dEps_btI_G, _Eps_btI_G, initial=0, axis=1)
ax_Diss.plot(_t_t_G * _t_F_scale, np.sum(Diss_btI_G[...,0], axis=0), alpha=1, label='G')

ax_omega.plot(_u_atI_F[0, :, 0], _omega_atI[0, :, 0])
ax_omega.set_xlabel(r'$\varepsilon$/-')
ax_omega.set_ylabel(r'$\omega$/-')
